In [1]:
import os
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [2]:
print('This notebook was run on ' + time.strftime("%d/%m/%Y") + ' at ' + time.strftime("%H:%M:%S") + '.')

This notebook was run on 02/01/2024 at 10:45:55.


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [4]:
dataset_directory = os.path.join(os.getenv('HOME'), 'icar-ng-data', 'dataset')
print('Dataset directory: {}'.format(dataset_directory))

dataset_file = 'dataset.csv'
print('Dataset file: {}'.format(dataset_file))

dataset_path = os.path.join(dataset_directory, dataset_file)
print('Dataset path: {}'.format(dataset_path))

# ======================================

model_directory = os.path.join(os.getenv('HOME'), 'icar-ng-data', 'model')
print('Model directory: {}'.format(model_directory))

model_file = 'cm2pixel_model.pt'
print('Model file: {}'.format(model_file))

model_path = os.path.join(model_directory, model_file)
print('Model path: {}'.format(model_path))

Dataset directory: /home/icar/icar-ng-data/dataset
Dataset file: dataset.csv
Dataset path: /home/icar/icar-ng-data/dataset/dataset.csv
Model directory: /home/icar/icar-ng-data/model
Model file: cm2pixel_model.pt
Model path: /home/icar/icar-ng-data/model/cm2pixel_model.pt


In [5]:
# Check whether dataset file exists
if not os.path.exists(dataset_path):
    print('Dataset file does not exist: {}'.format(dataset_path))
    exit()

# Check whether model directory exists
if not os.path.exists(model_directory):
    os.makedirs(model_directory)
    print('Created model directory: {}'.format(model_directory))

In [6]:
df_raw = pd.read_csv(dataset_path)
df_train = df_raw.sample(frac=0.8)
df_test = df_raw.drop(df_train.index)

min_x = np.min(df_train.iloc[:, 2:4].values, axis=0)
min_x = torch.tensor(min_x, dtype=torch.float32)
max_x = np.max(df_train.iloc[:, 2:4].values, axis=0)
max_x = torch.tensor(max_x, dtype=torch.float32)
min_y = np.min(df_train.iloc[:, 0:2].values, axis=0)
min_y = torch.tensor(min_y, dtype=torch.float32)
max_y = np.max(df_train.iloc[:, 0:2].values, axis=0)
max_y = torch.tensor(max_y, dtype=torch.float32)

print('min_x: {}, max_x: {}'.format(min_x, max_x))
print('min_y: {}, max_y: {}'.format(min_y, max_y))

min_x: tensor([   0.0000, -665.6403]), max_x: tensor([1200.0000,  848.5281])
min_y: tensor([ 36., 204.]), max_y: tensor([1272.,  692.])


In [7]:
class ForwardDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe
        x = self.dataframe.iloc[:, 2:4].values
        y = self.dataframe.iloc[:, 0:2].values
        self.x = torch.from_numpy(x).float()
        self.y = torch.from_numpy(y).float()
        
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [8]:
dataset_train = ForwardDataset(df_train)
dataset_test = ForwardDataset(df_test)
print('dataset_train: {}'.format(len(dataset_train)))
print('dataset_test: {}'.format(len(dataset_test)))

dataloader_train = DataLoader(dataset_train, batch_size=64, shuffle=True, pin_memory=True)
dataloader_test = DataLoader(dataset_test, batch_size=64, shuffle=True, pin_memory=True)

dataset_train: 118
dataset_test: 29


In [9]:
class MultiLayerPerceptron(nn.Module):
    def __init__(self, input_size, output_size, min_x, max_x, min_y, max_y):
        super(MultiLayerPerceptron, self).__init__()
        self.min_x = min_x
        self.max_x = max_x
        self.min_y = min_y
        self.max_y = max_y
        self.fc1 = nn.Linear(input_size, 4)
        self.fc2 = nn.Linear(4, 20)
        self.fc3 = nn.Linear(20, 80)
        self.fc4 = nn.Linear(80, 20)
        self.fc5 = nn.Linear(20, 4)
        self.fc6 = nn.Linear(4, output_size)

    def forward(self, x):
        x = (x - self.min_x) / (self.max_x - self.min_x)
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        x = self.fc6(x)
        x = x * (self.max_y - self.min_y) + self.min_y
        return x

In [10]:
model = MultiLayerPerceptron(2, 2, min_x.to(device), max_x.to(device), min_y.to(device), max_y.to(device)).to(device)

if os.path.exists(model_path):
    try:
        print('Loading model: {}'.format(model_path))
        model.load_state_dict(torch.load(model_path))
        print('Loaded model: {}'.format(model_path))
    except BaseException as e:
        print('Failed to load model: {}'.format(model_path))
        print(e)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

Loading model: /home/icar/icar-ng-data/model/cm2pixel_model.pt
Loaded model: /home/icar/icar-ng-data/model/cm2pixel_model.pt


In [11]:
min_test_loss = np.inf

for epoch in range(50000):
    model.train()
    train_loss = 0.0
    for i, (x, y) in enumerate(dataloader_train):
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        y_pred = model(x)
        loss = criterion(y_pred, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    if epoch % 100 == 0:
        model.eval()
        test_loss = 0.0
        for i, (x, y) in enumerate(dataloader_test):
            x = x.to(device)
            y = y.to(device)
            y_pred = model(x)
            loss = criterion(y_pred, y)
            test_loss += loss.item()

        save_model = False
        if test_loss < min_test_loss:
            min_test_loss = test_loss
            torch.save(model.state_dict(), model_path)
            save_model = True

        if save_model:
            print('epoch: {}, train_loss: {:.6f}, test_loss: {:.6f} (Saved)'.format(epoch, train_loss, test_loss))
        else:
            print('epoch: {}, train_loss: {:.6f}, test_loss: {:.6f}'.format(epoch, train_loss, test_loss))

epoch: 0, train_loss: 73125.289062, test_loss: 42636.726562 (Saved)
epoch: 100, train_loss: 8077.095215, test_loss: 4556.588379 (Saved)
epoch: 200, train_loss: 1718.268677, test_loss: 913.086914 (Saved)
epoch: 300, train_loss: 538.754898, test_loss: 314.804596 (Saved)
epoch: 400, train_loss: 219.228249, test_loss: 136.300308 (Saved)
epoch: 500, train_loss: 123.270710, test_loss: 81.291687 (Saved)
epoch: 600, train_loss: 89.402271, test_loss: 62.329597 (Saved)
epoch: 700, train_loss: 71.483967, test_loss: 54.795700 (Saved)
epoch: 800, train_loss: 62.338158, test_loss: 50.787575 (Saved)
epoch: 900, train_loss: 53.955643, test_loss: 47.993069 (Saved)
epoch: 1000, train_loss: 49.292027, test_loss: 45.540276 (Saved)
epoch: 1100, train_loss: 47.094807, test_loss: 43.074421 (Saved)
epoch: 1200, train_loss: 43.487545, test_loss: 40.875008 (Saved)
epoch: 1300, train_loss: 40.384619, test_loss: 39.035568 (Saved)
epoch: 1400, train_loss: 38.510887, test_loss: 37.528011 (Saved)
epoch: 1500, train_

In [12]:
onnx_path = os.path.join(model_directory, 'cm2pixel_model.onnx')
print('ONNX path: {}'.format(onnx_path))

try:
    print('Saving ONNX model: {}'.format(onnx_path))
    torch.onnx.export(model, torch.randn(1, 2).to(device), onnx_path, verbose=True)
    print('Saved ONNX model: {}'.format(onnx_path))
except BaseException as e:
    print('Failed to save ONNX model: {}'.format(onnx_path))
    print(e)

ONNX path: /home/icar/icar-ng-data/model/cm2pixel_model.onnx
Saving ONNX model: /home/icar/icar-ng-data/model/cm2pixel_model.onnx
Exported graph: graph(%onnx::Sub_0 : Float(1, 2, strides=[2, 1], requires_grad=0, device=cuda:0),
      %fc1.weight : Float(4, 2, strides=[2, 1], requires_grad=1, device=cuda:0),
      %fc1.bias : Float(4, strides=[1], requires_grad=1, device=cuda:0),
      %fc2.weight : Float(20, 4, strides=[4, 1], requires_grad=1, device=cuda:0),
      %fc2.bias : Float(20, strides=[1], requires_grad=1, device=cuda:0),
      %fc3.weight : Float(80, 20, strides=[20, 1], requires_grad=1, device=cuda:0),
      %fc3.bias : Float(80, strides=[1], requires_grad=1, device=cuda:0),
      %fc4.weight : Float(20, 80, strides=[80, 1], requires_grad=1, device=cuda:0),
      %fc4.bias : Float(20, strides=[1], requires_grad=1, device=cuda:0),
      %fc5.weight : Float(4, 20, strides=[20, 1], requires_grad=1, device=cuda:0),
      %fc5.bias : Float(4, strides=[1], requires_grad=1, device

In [13]:
model.eval()
for i, (x, y) in enumerate(dataloader_test):
    x = x.to(device)
    y = y.to(device)
    y_pred = model(x)
    rmse = torch.sqrt(torch.mean(torch.square(y - y_pred)))
    
    print('x\n{}'.format(x))
    print('y\n{}'.format(y))
    print('y_pred\n{}'.format(y_pred))
    print('y - y_pred\n{}'.format(y - y_pred))
    print('rmse: {:.6f}'.format(rmse))
    
    break
     

x
tensor([[ 474.3416,  158.1139],
        [ 200.0000,  300.0000],
        [ 789.1151, -131.5192],
        [ 450.0000,  200.0000],
        [ 200.0000, -250.0000],
        [ 447.2136,  223.6068],
        [ 450.0000,  300.0000],
        [ 100.0000,  -25.0000],
        [  25.0000,    0.0000],
        [  50.0000,  100.0000],
        [ 100.0000,  -75.0000],
        [ 848.5281, -848.5281],
        [ 100.0000, -125.0000],
        [ 450.0000, -300.0000],
        [ 500.0000,    0.0000],
        [  75.0000, -100.0000],
        [ 350.0000,  250.0000],
        [ 250.0000,  200.0000],
        [ 150.0000, -250.0000],
        [  75.0000,  -50.0000],
        [  25.0000,   50.0000],
        [  50.0000,  -50.0000],
        [   0.0000,    0.0000],
        [ 100.0000,  200.0000],
        [ 350.0000,  150.0000],
        [ 250.0000,  250.0000],
        [  50.0000,  -25.0000],
        [ 100.0000,  -50.0000],
        [ 350.0000, -200.0000]], device='cuda:0')
y
tensor([[ 464.,  270.],
        [  30.,  362.],
  

In [16]:
# Show all training data as table
pd.set_option('display.max_rows', len(df_train))
print(df_train)
# Export training data as CSV file
df_train.to_csv(os.path.join(model_directory, 'dataset_train_cm2pixel.csv'), index=False)

     pixel_x  pixel_y         cm_x        cm_y
59       872      260   450.000000 -200.000000
144     1132      286   353.553391 -353.553391
132      730      260   493.196962  -82.199494
141      986      272   416.025147 -277.350098
40      1272      342   200.000000 -300.000000
60       934      260   450.000000 -250.000000
69       444      532    50.000000   50.000000
125      338      214  1073.312629  536.656315
130      100      256   565.685425  565.685425
116      640      204  1200.000000    0.000000
77       562      446   100.000000   25.000000
75       300      486    75.000000  100.000000
101      110      336   250.000000  300.000000
56       698      262   450.000000  -50.000000
14      1244      520    50.000000 -150.000000
57       756      262   450.000000 -100.000000
43       822      324   250.000000 -100.000000
98       374      332   250.000000  150.000000
15       640      482    75.000000    0.000000
45      1000      324   250.000000 -200.000000
112      354 

In [17]:
# Show all test data as table
pd.set_option('display.max_rows', len(df_test))
print(df_test)
# Export test data as CSV file
df_test.to_csv(os.path.join(model_directory, 'dataset_test_cm2pixel.csv'), index=False)

     pixel_x  pixel_y        cm_x        cm_y
0        642      690    0.000000    0.000000
5        640      598   25.000000    0.000000
9        740      530   50.000000  -25.000000
10       838      528   50.000000  -50.000000
16       816      478   75.000000  -50.000000
17       984      474   75.000000 -100.000000
20       716      442  100.000000  -25.000000
21       794      440  100.000000  -50.000000
22       872      442  100.000000  -75.000000
24      1024      432  100.000000 -125.000000
33      1268      376  150.000000 -250.000000
39      1164      342  200.000000 -250.000000
52       924      284  350.000000 -200.000000
61       990      260  450.000000 -300.000000
66       412      598   25.000000   50.000000
71       252      532   50.000000  100.000000
84        46      452  100.000000  200.000000
95        30      362  200.000000  300.000000
99       286      332  250.000000  200.000000
100      204      336  250.000000  250.000000
104      432      294  350.000000 